In [25]:
import pandas as pd
import time
from concurrent.futures import ProcessPoolExecutor
import dask.dataframe as dd
start_full = time.time()


In [ ]:
# candidate_path = '/home/it/Desktop/RUN/Official/Source/CSAB_R1/Candidate.csv'
candidate_path = '/home/it/Desktop/RUN/Official/CSAB_R1/mergedCandidates.csv'
# choice_path = '/home/it/Desktop/RUN/Official/Source/CSAB_R1/Choices.csv'
choice_path = '/home/it/Desktop/RUN/Official/Source/CSAB_R1/Choices_NIC.csv'
seats_path = '/home/it/Desktop/RUN/Official/Source/CSAB_R1/Seat.csv'
# seats_path = '/home/it/Desktop/RUN/Official/Source/CSAB_R1/seats_new.csv'

last_can_confirm_path = '/home/it/Desktop/RUN/Official/CSAB_R1/last_op_append_candidates.csv'
last_can_confirm = pd.read_csv(last_can_confirm_path)
last_can_confirm_rolls = last_can_confirm['Rollno']

candidate = pd.read_csv(candidate_path)
choice = pd.read_csv(choice_path)
choice_filtered = choice[choice['Validity'] != 'N']
seat = pd.read_csv(seats_path)

In [ ]:
# RollNo,ChoiceNo,Instcd,Brcd,Validity
len(choice['RollNo'].unique())

In [28]:
temp_cand = candidate[candidate['InstCd']>=200]

In [ ]:
temp_cand

In [ ]:
temp_cand['InstCd'].dtype

In [ ]:
temp_cand['InstCd'].dtype


In [ ]:
temp_cand['InstCd']=temp_cand['InstCd'].astype(int)

In [33]:
# new_rows = pd.DataFrame({
#     'RollNo': temp_cand['RollNo'],
#     'ChoiceNo': temp_cand['Max_Choice'] + 1,
#     'Instcd': temp_cand['InstCd'],
#     'Brcd': temp_cand['BrCd'],
#     'Validity': "NULL"
# })

# # Append the new rows to choice_filtered
# choice_filtered = pd.concat([choice_filtered, new_rows], ignore_index=True)

In [34]:
# for index, row in temp_cand.iterrows():
#     if row['InstCd']>0:
#         new_row = {'RollNo': row['RollNo'], 'ChoiceNo': row['Max_Choice']+1, 'Instcd': row['InstCd'], 'Brcd': row['BrCd'], 'Validity': "NULL"}
#         # choice_filtered = choice_filtered.append(new_row, ignore_index=True)
#         choice_filtered.loc[len(choice_filtered)] = new_row

In [ ]:
start_ti = time.time()
df = pd.merge(choice_filtered, candidate, on="RollNo")
df.to_csv('candidate_choice_merge_CSAB0.csv', index=False)
end_ti = time.time()
print(f"time taken to merge candidate and choice and save as csv- {end_ti- start_ti}")

In [36]:
# df = pd.read_csv('/home/it/Desktop/RUN/Official/CSAB_R1/candidate_choice_merge_CSAB1.csv')
# df['Cat'].unique()

In [ ]:
df['Gender'].unique()

In [38]:
institutes={}
for index, row in seat.iterrows():
    if row['Quota']=='OS' or row['Quota']=='HS':
        if not row['InstCd'] in institutes.keys():
            institutes[row['InstCd']]=[row['Stcode1'],row['Stcode2'],row['Stcode3'],row['Stcode4'],row['Stcode5'],row['Stcode6'],row['Stcode7'],row['Stcode8'],row['Stcode9'],row['Stcode10']]

In [39]:
institute_quota_dict = {}

for _, row in seat.iterrows():
    institute = row['InstCd']
    quota = row['Quota']
    
    if institute not in institute_quota_dict:
        institute_quota_dict[institute] = set()
    
    institute_quota_dict[institute].add(quota)

# Process the dictionary to set the final quota values
final_institute_quota_dict = {}

for institute, quotas in institute_quota_dict.items():
    if quotas == {'AI'}:
        final_institute_quota_dict[institute] = 'AI'
    elif quotas == {'AI', 'HS'}:
        final_institute_quota_dict[institute] = 'AH'
    else:
        final_institute_quota_dict[institute] = 'OH'

In [40]:
del df, candidate, choice

In [ ]:
# ['BC', 'GN', 'SC', 'EW', 'ST']
def determine_virtual_categories(row):
    if row['Cat'] == 'BC':
        return ['OP', 'BC']
    elif row['Cat'] == 'GN':
        return ['OP']
    elif row['Cat'] == 'EW':
        return ['OP', 'EW']
    elif row['Cat'] == 'SC':
        return ['OP', 'SC']
    elif row['Cat'] == 'ST':
        return ['OP', 'ST']
    else:
        return []

def determine_virtual_sub_categories(row):
    if row['PwD'] == 2:
        return ['NO']
    else:
        return ['NO', 'PH']

def determine_virtual_gender(row):
    if row['Gender'] == 2:
        return ['F', 'B']
    else:
        return ['B']


def determine_virtual_state_quota(row):
    if final_institute_quota_dict[row['Instcd']] == 'AI':
    # if row['InstQuota'] == 'AI':
        return ['AI']

    if row['Instcd'] == 209 and row['SCode']=='GO':
        return ['AI', 'GO']    
    
    if row['Instcd'] == 225 and row['SCode']=='JK':
        return ['AI', 'JK']    
    
    if row['Instcd'] == 225 and row['SCode']=='LA':
        return ['AI', 'LA']    
    state_codes = institutes[row['Instcd']]
    if row['SCode'] in state_codes:
        return ['AI', 'HS']
    else:
        # if row['InstQuota'] == 'OH':
        if final_institute_quota_dict[row['Instcd']] == 'OH':
            return ['AI', 'OS']
        else:
            return ['AI']


def process_chunk(chunk):
    chunk = chunk[
        # (chunk['RStatus'] == 'RP') & 
        # (chunk['Withdraw'] == 'N') & 
        # (chunk['Validity'] != 'N') &    
        (
            (chunk['Decision'] == 'FL') | 
            ((chunk['Decision'] == 'FR') & (chunk['ChoiceNo'] >= chunk['AllotedOpt'])) | 
            ((chunk['Decision'] == 'SL') & 
            ((chunk['ChoiceNo'] >= chunk['AllotedOpt']) | ((chunk['ChoiceNo'] < chunk['AllotedOpt']) & chunk['InstCd'].notna() & (chunk['InstCd'].fillna(-1).astype(int) == chunk['Instcd']))))
        )
    ]
    chunk['Cat_V'] = chunk.apply(determine_virtual_categories, axis=1)
    chunk = chunk.explode('Cat_V')
    chunk['SubCat_V'] = chunk.apply(determine_virtual_sub_categories, axis=1)
    chunk = chunk.explode('SubCat_V')
    chunk['State_quota_V'] = chunk.apply(determine_virtual_state_quota, axis=1)
    chunk = chunk.explode('State_quota_V')
    chunk['Gender_V'] = chunk.apply(determine_virtual_gender, axis=1)
    chunk = chunk.explode('Gender_V')
    # chunk['State_quota_V'] = chunk.apply(determine_virtual_state_quota, axis=1)
    # chunk = chunk.explode('State_quota_V')
    return chunk

start_time_loading_merging = time.time()

chunks = pd.read_csv(r'candidate_choice_merge_CSAB0.csv', chunksize=70000)

processed_chunks = []

with ProcessPoolExecutor() as executor:
    for processed_chunk in executor.map(process_chunk, chunks):
        processed_chunks.append(processed_chunk)

merged_df = pd.concat(processed_chunks, ignore_index=True)

end_time_loading_merging = time.time()
print(f"Time taken for loading tables and merging and running virtulization: {end_time_loading_merging - start_time_loading_merging:.2f} seconds")


print(f"Now Sorting:")
merged_df.sort_values(by=['RollNo', 'ChoiceNo'], inplace=True)
end_time_sorting = time.time()

print(f"Time taken for Sorting: {end_time_sorting - end_time_loading_merging:.2f} seconds")


print(f"Now creating Virtual count")
merged_df['ChoiceNo_V'] = merged_df.groupby(['RollNo']).cumcount() + 1
end_time_V = time.time()
print(f"Time taken for creating Virtual count: {end_time_V- end_time_sorting:.2f} seconds")


# # Ensure Adv_DS_V column is empty initially
# merged_df['Adv_DS_V'] = None

# # Identify RollNos with Adv_DS == 1.0
# adv_ds_rollnos = merged_df[merged_df['Adv_DS'] == 1.0]['RollNo'].unique()

# ds_df = merged_df[merged_df['RollNo'].isin(adv_ds_rollnos)]
# normal_df = merged_df[~merged_df['RollNo'].isin(adv_ds_rollnos)]

# # Create new rows for DS RollNos at the end of each choice
# new_rows = []
# for rollno in adv_ds_rollnos:
#     unique_choices = ds_df[ds_df['RollNo'] == rollno]['ChoiceNo'].unique()
#     for choice in unique_choices:
#         choice_rows = ds_df[(ds_df['RollNo'] == rollno) & (ds_df['ChoiceNo'] == choice)]
#         if choice_rows.iloc[0]['InstCode'] < 200:  # Only add DS rows for IITs
#             last_virtual_choice_no = choice_rows['ChoiceNo_V'].max() + 1
#             last_row = choice_rows.iloc[-1]
#             new_row = last_row.copy()
#             new_row['ChoiceNo_V'] = last_virtual_choice_no
#             new_row['Adv_DS_V'] = 1
#             new_rows.append(new_row)

# new_rows_df = pd.DataFrame(new_rows)

# # Append new rows to the DS DataFrame
# ds_final_df = pd.concat([ds_df, new_rows_df], ignore_index=True)

# # Recalculate VirtualChoiceNumber to account for new DS rows
# ds_final_df.sort_values(by=['RollNo', 'ChoiceNo'], inplace=True)
# ds_final_df['ChoiceNo_V'] = ds_final_df.groupby(['RollNo']).cumcount() + 1

# Select columns to reorder them as needed
reordered_columns = ['RollNo', 'Instcd', 'Brcd', 'ChoiceNo', 'ChoiceNo_V', 'Cat_V', 'SubCat_V', 'Gender_V', 'State_quota_V']

normal_df = merged_df[reordered_columns]
# ds_final_df = ds_final_df[reordered_columns]

print(f"Now creating csv file")
normal_df.to_csv('Normal_Virtualized_Choices_CSAB0.csv', index=False)
# ds_final_df.to_csv('DS_Virtualized_Choices_R7.csv', index=False)


print(f"Time taken for creating csv file: {time.time() - end_time_V:.2f} seconds")

In [ ]:
final_institute_quota_dict

In [ ]:
end_full = time.time()
total_time= end_full -start_full
print(f"Total time taken  - {total_time}")
print(f" In minutes - {total_time/60}")

In [ ]:
normal_df['Gender_V'].unique()

In [ ]:
normal_df[normal_df['RollNo']==240310000005]
